In [300]:
import pandas as pd

In [301]:
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords

morph = MorphAnalyzer()
stopwords_ru = stopwords.words("russian")

def morphe(str):
    return ' '.join([morph.normal_forms(w)[0] for w in str.split() if w not in stopwords_ru])

def morphem(df):
    from nltk.corpus import stopwords
    from pymorphy3 import MorphAnalyzer
    morph = MorphAnalyzer()
    stopwords_ru = stopwords.words("russian")
    df = df.copy()
    df['query'].apply(lambda str:' '.join([morph.normal_forms(w.lower())[0] for w in str.split() if w not in stopwords_ru]))
    return df

In [302]:
fillna_columns = [
    "занятость",
    "по должности-лемме",
    "по дополнительному признаку",
    "по условиям",
    "общие фразы",
]

In [303]:
df = pd.read_csv("answers.csv", index_col='Unnamed: 0')
for col in fillna_columns:
    df.fillna({col: f'{col}_нет'}, inplace=True)

In [304]:
onehot_columns = ['по должности-лемме', 'общие фразы']
multilabel_columns = ['занятость', 'по дополнительному признаку',  'по условиям']
vectorizer_columns = 'query'

In [305]:
def split(x):
    return [i for i in set([x.lower() for x in x.split(',')]) if len(i)>0]

for col in multilabel_columns:
    df[col] = df[col].apply(lambda x: split(x))

In [306]:
X = df[['query']]
y = df.drop(columns=['query'])

In [307]:
y

,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
1,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
2,[занятость_нет],Рабочий,[по дополнительному признаку_нет],[по условиям_нет],общие фразы_нет
3,[на неполный день],Водитель,[по дополнительному признаку_нет],[по условиям_нет],общие фразы_нет
4,[занятость_нет],Водитель,[по дополнительному признаку_нет],[по условиям_нет],общие фразы_нет
...,...,...,...,...,...
14240,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
14241,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
14242,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
14243,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза


In [308]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import CountVectorizer


In [309]:
from sklearn.pipeline import FunctionTransformer


preprocessor_y = ColumnTransformer(
    transformers=[

        (
            "onehot",
            OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
            onehot_columns,
        ),
        ('multilabel0', CountVectorizer(analyzer=set), multilabel_columns[0]),
        ("multilabel1", CountVectorizer(analyzer=set), multilabel_columns[1]),
        ("multilabel2", CountVectorizer(analyzer=set), multilabel_columns[2]),
        # ("vectorizer", CountVectorizer(), vectorizer_columns),
    ]
)

preprocessor_x = Pipeline([
    ('morphem', FunctionTransformer(morphem)),
    ('vectorizer', ColumnTransformer([('vectorize', CountVectorizer(), vectorizer_columns)])),
])

In [310]:
from sklearn.base import BaseEstimator, TransformerMixin


class InverseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, pipe):
        self.pipe = pipe

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        arr = X[0]
        inverse_transformers = self.pipe.transformers_

        len1 = len(inverse_transformers[0][1].get_feature_names_out())
        len2 = len(list(inverse_transformers[1][1].vocabulary_.keys()))
        len3 = len(list(inverse_transformers[2][1].vocabulary_.keys()))

        res = inverse_transformers[0][1].inverse_transform([arr[:len1]])
        res2 = inverse_transformers[1][1].inverse_transform([arr[len1 : len1 + len2]])
        res3 = inverse_transformers[2][1].inverse_transform([arr[len1 + len2 : len1 + len2 + len3]])
        res4 = inverse_transformers[3][1].inverse_transform([arr[len1 + len2 + len3:]])

        output = dict(zip(inverse_transformers[0][1].feature_names_in_, res[0]))
        output["занятость"] = res2[0].tolist()
        output["по дополнительному признаку"] = res3[0].tolist()
        output["по условиям"] = res4[0].tolist()

        for key, value in output.items():
            if value is None:
                output[key] = 'Нет'
            if isinstance(value, list):
                for idx, item in enumerate(value):
                    output[key][idx] = item.replace(f'{key}_нет', 'Нет')
            else:
                output[key] = output[key].replace(f'{key}_нет', 'Нет')
        return output

In [311]:
from sklearn.tree import DecisionTreeClassifier

In [312]:
# pipeline = Pipeline([
#     ("preprocessor", preprocessor),
#     ("model", MLPClassifier(max_iter=400)),
#     ("inverse", FunctionTransformer())
# ])

In [313]:
y_prep = preprocessor_y.fit_transform(y)
y_prep

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [314]:
# x_prep = preprocessor_x.fit_transform(X)
# x_prep.todense()

In [315]:
# model = MLPClassifier(max_iter=400)
model = DecisionTreeClassifier()

In [316]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score

models = [
    
    MultiOutputClassifier(DecisionTreeClassifier()), #0.8177774309672764
    MLPClassifier(max_iter=400)
]


In [317]:
from sklearn.model_selection import cross_val_score

In [318]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor_x),
    ("model", model),
    # ("inverse", InverseTransformer(preprocessor_y))
])

inverser = InverseTransformer(preprocessor_y)

In [319]:
import time

bestpipe = None
bestscore = 0

for m in models:
    pipeline2 = Pipeline(
        steps=[
            ("preprocessor", preprocessor_x),
            ("model", m),
            # ("inverse", InverseTransformer(preprocessor_y))
        ]
    )
    start_time = time.time()

    try:
        # score = cross_val_score(pipeline2, X, y_prep, cv=4, n_jobs=-1)
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_prep, test_size=0.15, random_state=42)
        
        # Fit the model
        pipeline2.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = pipeline2.predict(X_test)
        
        # Calculate the score
        if y_test.ndim == 1 or y_test.shape[1] == 1:
            score = accuracy_score(y_test, y_pred)  # For classification
        else:
            score = r2_score(y_test, y_pred)  # For regression
    except Exception as e:
        print('----------------')
        print(m)
        print(e)
        print('----------------')
        continue

    end_time = time.time()
    loop_time = end_time - start_time

    print(f'Модель: {m}\nТочность: {score}\nВремя: {loop_time}\n\n')
    if score > bestscore:
        bestscore = score
        bestpipe = pipeline2

Модель: MultiOutputClassifier(estimator=DecisionTreeClassifier())
Точность: 0.8238507437836979
Время: 15.887998104095459


Модель: MLPClassifier(max_iter=400)
Точность: 0.8173370069443464
Время: 585.7514081001282




In [320]:
# pipeline.fit(X, y_prep)

In [336]:
example_qeury = "работа бухгалтером с ежедневной оплатой без опыта удаленно"

frame = pd.DataFrame({'query': [example_qeury]})
preds = bestpipe.predict(frame)

In [337]:
# example_prep = preprocessor_x.transform(frame)
# preds = model.predict(example_prep)
# preds = pipeline.predict(frame)
result = inverser.transform(preds)
# cut = preds[0]
result

{'по должности-лемме': 'Бухгалтер',
 'общие фразы': 'Нет',
 'занятость': ['удаленная'],
 'по дополнительному признаку': ['без опыта'],
 'по условиям': ['с ежедневной оплатой']}

In [323]:
# from joblib import dump
# dump(pipeline, 'model.joblib')

In [338]:
import dill

with open('model.pkl', 'wb') as file:
    pipe_data = {
        'pipeline': bestpipe,
        'inverser': inverser
    }
    dill.dump(pipe_data, file)

In [325]:
# inverse_transformers = preprocessor_y.transformers_

In [326]:
# list(inverse_transformers[1][1].vocabulary_.keys())

In [327]:
# res = inverse_transformers[0][1].inverse_transform([cut[:303]])
# res

In [328]:
# output = dict(zip(inverse_transformers[0][1].feature_names_in_, res[0]))

In [329]:
# res2 = inverse_transformers[1][1].inverse_transform([cut[303:316]])
# output['занятость'] = res2[0].tolist()

In [330]:
# res2

In [331]:
# res3 = inverse_transformers[2][1].inverse_transform([cut[316:]])
# output['по дополнительному признаку'] = res3[0].tolist()

In [332]:
# output